In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 https

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Software_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17747349|R2EI7QLPK4LF7U|B00U7LCE6A|     106182406|CCleaner Free [Do...|Digital_Software|          4|            0|          0|   N|                Y|          Four Stars|      So far so good| 2015-08-31|
|         US|   10956619|R1W5OMFK1Q3I3O|B00HRJMOM4|     162269768|ResumeMaker Profe...|Digital_Software|          3|    

In [5]:
# Create vine dataframe
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2EI7QLPK4LF7U|          4|            0|          0|   N|                Y|
|R1W5OMFK1Q3I3O|          3|            0|          0|   N|                Y|
| RPZWSYWRP92GI|          1|            1|          2|   N|                Y|
|R2WQWM04XHD9US|          5|            0|          0|   N|                Y|
|R1WSPK2RA2PDEF|          4|            1|          2|   N|                Y|
|R11JVGRZRHTDAS|          5|            4|          4|   N|                N|
|R2B8468OKXXYE2|          1|            1|          1|   N|                N|
|R2HGGCCZSSNUCB|          1|            0|          0|   N|                Y|
| REEE4LHSVPRV9|          1|            0|          0|   N|                Y|
|R25OMUUILFFHI9|          5|            0|          0|   N|     

In [6]:
# Only select reviews with votes greater than or equal to 20
from pyspark.sql.functions import col
helpful_vine_df = vine_df.filter(col('total_votes') >= 20)
helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXKYQDUACCRIA|          2|           20|         20|   N|                N|
|R10Q403A12Q9PX|          1|          123|        136|   N|                N|
|R2OH6106YSNSV4|          1|           20|         24|   N|                Y|
|R3C1OKM81KRDMO|          4|          159|        167|   N|                Y|
|R180C1J73S8MCV|          1|           27|         31|   N|                N|
|R208352XIVG6CL|          2|           17|         25|   N|                N|
|R1J4DGLB9OKW2B|          4|            9|         20|   N|                N|
|R2G3JWLCDAPN5I|          5|            9|         21|   N|                N|
| RXM58ZCC2N93C|          1|           17|         23|   N|                N|
| RJSGOYKD6T7KY|          5|            3|         28|   N|     

In [7]:
# Select reviews where number of helpful_votes / total_votes >= 0.5
filtered_vine_df = helpful_vine_df.filter(col('helpful_votes') / col('total_votes') >= 0.5)
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXKYQDUACCRIA|          2|           20|         20|   N|                N|
|R10Q403A12Q9PX|          1|          123|        136|   N|                N|
|R2OH6106YSNSV4|          1|           20|         24|   N|                Y|
|R3C1OKM81KRDMO|          4|          159|        167|   N|                Y|
|R180C1J73S8MCV|          1|           27|         31|   N|                N|
|R208352XIVG6CL|          2|           17|         25|   N|                N|
| RXM58ZCC2N93C|          1|           17|         23|   N|                N|
| RC9E2K8LLD6P6|          1|           14|         20|   N|                N|
|R2PSBYN2BGZQPO|          1|           28|         32|   N|                Y|
| RE1GOM9XMXJ3J|          5|           18|         24|   N|     

In [8]:
# Select reviews that were part of the Vine program
paid_vine_df = filtered_vine_df.filter(col('vine') == 'Y')
paid_vine_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [9]:
# Select reviews not a part of the Vine program
unpaid_vine_df = filtered_vine_df.filter(col('vine') == 'N')
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXKYQDUACCRIA|          2|           20|         20|   N|                N|
|R10Q403A12Q9PX|          1|          123|        136|   N|                N|
|R2OH6106YSNSV4|          1|           20|         24|   N|                Y|
|R3C1OKM81KRDMO|          4|          159|        167|   N|                Y|
|R180C1J73S8MCV|          1|           27|         31|   N|                N|
|R208352XIVG6CL|          2|           17|         25|   N|                N|
| RXM58ZCC2N93C|          1|           17|         23|   N|                N|
| RC9E2K8LLD6P6|          1|           14|         20|   N|                N|
|R2PSBYN2BGZQPO|          1|           28|         32|   N|                Y|
| RE1GOM9XMXJ3J|          5|           18|         24|   N|     

In [10]:
# Calculate summary info
from pyspark.sql.functions import when, count, lit

ratings_total_df = filtered_vine_df.groupBy('vine').agg(
    count(col('vine')).alias('Total Reviews'),
    count(when(col('star_rating') == 5, True)).alias('Total 5 Star Reviews'),
    (count(when(col('star_rating') == 5, True)) / (count(col('vine')))*100).alias('% 5 Star Reviews')
).show()

+----+-------------+--------------------+-----------------+
|vine|Total Reviews|Total 5 Star Reviews| % 5 Star Reviews|
+----+-------------+--------------------+-----------------+
|   N|         2120|                 314|14.81132075471698|
+----+-------------+--------------------+-----------------+

